<a href="https://colab.research.google.com/github/Maxouuu4/test_git/blob/main/math0487_nb1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📓 Notebook 1 : **Statistiques descriptives**

## Objectifs 🎯

Les objectifs de ce notebook sont les suivants :
- *explorer* un jeu de données et en *extraire* différentes statistiques descriptives ;
- *représenter* les données en utilisant plusieurs outils ;
- *visualiser* les propriétés de convergence de certaines statistiques.

## Contexte 🔍

Dans ce premier notebook, nous allons aborder différents concepts liés aux statistiques descriptives à l'aide d'un ensemble de données disponible [ici](https://www.kaggle.com/datasets/mosapabdelghany/medical-insurance-cost-dataset). Cet ensemble de données contient des informations sur les coûts d'assurance médicale pour 1 338 personnes. Il comprend des variables démographiques et liées à la santé telles que l'âge, le sexe, l'IMC, le nombre d'enfants, le statut tabagique, la région de résidence aux États-Unis et les frais (_charges_), qui représentent le coût de l'assurance médicale facturé à la personne.

## Importation des librairies Python 🐍
Commençons par importer quelques librairies nécessaires pour la manipulation de données :
- [Kaggle](https://www.kaggle.com/) est une plateforme de Data Science de laquelle nous allons télécharger une base de données. Nous utilisons pour ce faire son API publique.
- [Matplotlib](https://matplotlib.org/) est la librairie la plus répandue pour visualiser des données en Python.
- [pandas](https://pandas.pydata.org/) est la librairie la plus répandue pour manipuler et analyser des données en Python.
- [NumPy](https://numpy.org/) est le package fondamental pour le calcul scientifique en Python.
- [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/stable/) sont des utilitaires de contrôle interactif pour navigateur, que nous allons utiliser pour créer des visualisations interactives.

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import ipywidgets as widgets
from ipywidgets import Button, Output, VBox, HBox
from IPython.display import display

from google.colab import data_table, output
output.enable_custom_widget_manager()
data_table.enable_dataframe_formatter()

## Chargement des données 🐼

Nous allons dans un premier temps charger les données depuis Kaggle et les stocker dans une `DataFrame` de la librairie pandas. Une `DataFrame` constitue la structure de données primaire de pandas, et consiste en une table à deux dimensions, de taille modifiable, contenant des données potentiellement hétérogènes. Elle contient des axes labélisés (lignes et colonnes), et des opérations arithmétiques peuvent être réalisées sur ces axes. Voyons ensemble à quoi cela ressemble ! 👀


In [ ]:
# Path to file for storing the data
file_path = "insurance.csv"

# Creation of the DataFrame using downloaded data
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "mosapabdelghany/medical-insurance-cost-dataset",
  file_path,
)

# Display the first five elements of the DataFrame
print("5 premiers éléments :\n")
df.head(5)

Using Colab cache for faster access to the 'medical-insurance-cost-dataset' dataset.
5 premiers éléments :



,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


Voilà, ce n'était pas bien compliqué. 😉 Nous avons donc une table contenant les données de notre *échantillon*. Nous voyons que, pour plusieurs individus, nous avons accès à différentes variables : l'age, le nombre d'enfants, etc. Nous pourrions également vouloir avoir un résumé de cet échantillon, contenant par exemple sa taille :

In [ ]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Visualisation des données 📊

Bien, maintenant que nous avons téléchargé les données, la première étape d'une étude statistique est de visualiser ces fameuses données ! Et de manière plus parlante que sous la forme d'une table pandas ... Concentrons-nous sur une seule variable (les _charges_), et voyons comment les données sont réparties !

Pour ce faire, trois outils sont particulièrement adéquats :
- l'histogramme,
- la fonction de répartition empirique (_empirical cumulative density function_, ECDF),
- la boîte à moustaches (_boxplot_).

Ces trois visualisations qui en découlent sont certes différentes, mais elles représentent les mêmes données !

> ⚠️
Vous avez l'occasion de varier la taille de l'échantillon considéré ; en pratique, on travaillera plutôt avec toutes les données à notre disposition, mais pour la compréhension de certains concepts, nous considérerons ici des échantillons "artificiels" de l'échantillon de départ. C'est une manière détournée de faire _comme si_ on connaissait les rouages du procédé générant les données à notre disposition, et donc _comme si_ on pouvait générer des données. **Soyons bien clairs**, le but d'une étude statistique est notamment de modéliser un procédé générateur de données afin d'en tirer des conclusions, et ce à partir de **données collectées**. Connaître le procédé générateur rendrait évidemment la collecte de données complètement inutile...


Voyons ensemble comment avoir une idée plus visuelle de la répartition de nos données. 👀

In [ ]:
# Get charges data
charges = df['charges'].values

# Initial values
init_sample_size = len(df)
init_bins = 30

# Sliders
slider_sample_size = widgets.IntSlider(value=init_sample_size, min=1, max=len(df), step=1, description='Sample size:' )
slider_bins = widgets.IntSlider(value=init_bins, min=10, max=100, step=1, description='Bins:')

# Update function
def update(sample_size, bins):
    # Draw new sample
    sample_df = df.sample(n=sample_size, replace=True)

    # Figure and axes
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 6))

    # Histogram
    hist_vals = ax1.hist(sample_df['charges'], bins=init_bins, density=True, alpha=0.7, label='Histogram')
    ax1.set_title('Histogram of Charges')
    ax1.set_xlabel('Charges')
    ax1.set_ylabel('Density')

    # Empirical CDF
    sorted_charges = np.sort(sample_df['charges'])
    ecdf = np.arange(1, len(sorted_charges) + 1) / len(sorted_charges)
    (line_ecdf,) = ax2.plot(sorted_charges, ecdf, label='Empirical CDF')
    ax2.set_title('Empirical CDF of Charges')
    ax2.set_xlabel('Charges')
    ax2.set_ylabel('ECDF')

    # Boxplot
    box = ax3.boxplot(sample_df['charges'], vert=False, patch_artist=True)
    ax3.set_title('Boxplot of Charges')
    ax3.set_xlabel('Charges')

    plt.show()

# Link widgets to the plotting function
interactive_reparam_plot = widgets.interactive(
    update,
    sample_size=slider_sample_size,
    bins=slider_bins,
)

# Display the widgets and the plot
display(interactive_reparam_plot)

interactive(children=(IntSlider(value=1338, description='Sample size:', max=1338, min=1), IntSlider(value=30, …

Deux curseurs sont à votre disposition : le premier change la taille du sous-échantillon que l'on considère, et le second modifie le nombre de "bins" utilisées pour générer l'histogramme. N'hésitez pas à jouer avec afin de voir quelle influence ont ces facteurs sur les représentations visuelles que nous avons tracées !

❓
Plusieurs normalisations existent pour un histogramme. Quelle convention est ici utilisée ? Pouvez-vous modifier le code Python pour utiliser une autre convention ?

❓
Que représentent les moustaches de la boîte ? Quid des limites de la boîte, et de la ligne orange ? Pouvez-vous mettre cela en relation avec les valeurs de la table donnée par la fonction `.describe()` utilisée plus haut ?

❓
Que représentent les cercles blancs sur cette même figure ? Comment leur nombre évolue-t-il avec la taille de l'échantillon ? Pourquoi ? Pour une même taille d'échantillon, aurez-vous les mêmes cercles ?

❓
Que représente l'ECDF ? Comment est-elle construite ?

❓
Une fois ces représentations visuelles générées, comment peut-on s'en servir pour poursuivre notre étude statistique ? Qu'entend-on alors par _inférence_ ?



> 🧠 Écrivez ici ce que vous en pensez !


> 🤔 Les représentations utilisées tant que maintenant ne se concentrent que sur une seule variable. Cependant, considérer plusieurs variables ensemble, et ainsi observer leur distribution d'échantillon jointe, peut apporter d'autres informations.

Une première manière de visualiser un couple de données est de simplement réaliser un _scatter plot_ des deux variables, ou encore un histogramme à trois dimensions. 👀

In [ ]:
# Get the list of numerical columns for the dropdown menus
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()

# Create dropdown widgets for selecting variables
dropdown_x = widgets.Dropdown(options=numerical_cols, description='Variable X:')
dropdown_y = widgets.Dropdown(options=numerical_cols, description='Variable Y:')

# Output widget for the plots
out_plots = Output()

# Update function to generate plots
def update_plots(x_var, y_var):
    with out_plots:
        out_plots.clear_output(wait=False)

        # Create figure and axes
        fig = plt.figure(figsize=(12, 6))
        ax1 = fig.add_subplot(121) # 2D scatter plot
        ax2 = fig.add_subplot(122, projection='3d') # 3D histogram

        # 2D Scatter Plot
        ax1.scatter(df[x_var], df[y_var])
        ax1.set_xlabel(x_var)
        ax1.set_ylabel(y_var)
        ax1.set_title(f'Scatter Plot of {x_var} vs {y_var}')

        # 3D Histogram
        hist, xedges, yedges = np.histogram2d(df[x_var], df[y_var], bins=30)
        xpos, ypos = np.meshgrid(xedges[:-1] + 0.5*(xedges[1] - xedges[0]), yedges[:-1] + 0.5*(yedges[1] - yedges[0]), indexing="ij")
        xpos = xpos.ravel()
        ypos = ypos.ravel()
        zpos = 0
        dx = dy = 0.5 * (xedges[1] - xedges[0])
        dz = hist.ravel()

        ax2.bar3d(xpos, ypos, zpos, dx, dy, dz, zsort='average')
        ax2.set_xlabel(x_var)
        ax2.set_ylabel(y_var)
        ax2.set_zlabel('Frequency')
        ax2.set_title(f'3D Histogram of {x_var} and {y_var}')

        plt.tight_layout()
        plt.show()

# Link dropdowns to the update function
interactive_plots = widgets.interactive(update_plots, x_var=dropdown_x, y_var=dropdown_y)

# Display the widgets and plots
display(VBox([HBox([dropdown_x, dropdown_y]), out_plots]))

En fonction du couple de variable considéré, ces deux types de visualisation ne donnent pas toujours des représentations très parlantes.

❓Le scatter plot permet cependant d'identifier facilement un lien entre deux variables. Comment se nomme ce lien ?  Indique-t-il qu'une des variables à un effet causal sur l'autre ?

❓Certaines variables n'ont pas été prises en compte, par exemple le fait que l'individu fume ou pas. Comment pourrions-nous les prendre en compte dans notre visualisation ? En terme de visualisation, prendre en compte le nombre d'enfants comme variable était-il une bonne idée ?

> 🧠 Écrivez ici ce que vous en pensez !

Une manière plus visuelle de représenter la distribution d'un couple de variables dans l'échantillon est d'utiliser une échelle de couleur pour représenter la densité dans un plan à deux dimensions. 👀

In [ ]:
# Get the list of numerical columns for the dropdown menus
numerical_cols = df.select_dtypes(include=np.number).columns.tolist()

# Create dropdown widgets for selecting variables
dropdown_x = widgets.Dropdown(options=numerical_cols, description='Variable X:')
dropdown_y = widgets.Dropdown(options=numerical_cols, description='Variable Y:')

# Output widget for the plot
out_heatmap = Output()

# Update function to generate the heatmap
def update_heatmap(x_var, y_var):
    with out_heatmap:
        out_heatmap.clear_output(wait=True)

        # Create figure and axes
        fig, ax = plt.subplots(figsize=(8, 6))

        # 2D Histogram (for heatmap)
        hist, xedges, yedges = np.histogram2d(df[x_var], df[y_var], bins=30)
        X, Y = np.meshgrid(xedges[:-1], yedges[:-1], indexing="ij")

        # Plot heatmap
        im = ax.pcolormesh(X, Y, hist, cmap='viridis')
        fig.colorbar(im, ax=ax, label='Frequency')

        ax.set_xlabel(x_var)
        ax.set_ylabel(y_var)
        ax.set_title(f'Heatmap Histogram of {x_var} vs {y_var}')

        plt.show()

# Link dropdowns to the update function
interactive_heatmap = widgets.interactive(update_heatmap, x_var=dropdown_x, y_var=dropdown_y)

# Display the widgets and heatmap
display(VBox([HBox([dropdown_x, dropdown_y]), out_heatmap]))

## Extraction de statistiques ⛏️

Après avoir visualisé les données d'un échantillon, on en extrait habituellement des statistiques. Nous avons déjà extrait les statistiques les plus courantes (moyenne, médiane, écart-type, quartiles) pour les différentes variables de notre échantillon à l'aide de la fonction `.describe()` de pandas.

Pour bien comprendre la notion de statistique d'échantillon, nous allons cependant à nouveau générer des sous-échantillons à partir de l'échantillon de départ. Une statistique, par exemple la _moyenne d'échantillon_ $\bar{Y}$, est une variable aléatoire qui dépend des variables aléatoires $Y_j$ d'un échantillon, qui va donc prendre une valeur $\bar{y}$ différente (on parle de _cristallisation_) pour chaque échantillon. Ainsi, si l'on considère de nombreux échantillons différents, on observera de nombreuses cristallisations différentes de cette moyenne d'échantillon, et donc une _distribution_ des valeurs de la moyenne d'échantillon.

Calculons des statistiques pour différents sous-échantillons (que vous pouvez générer en appuyant sur le bouton `Sample`), et visualisons comment elles se distribuent. 👀

In [ ]:
sample_size = 500  # fixed sample size
charges = df['charges'].values
full_sample_mean = charges.mean()
full_sample_std = charges.std()

# Storage for statistics
means = []
vars = []
norm_means = []

# Outputs
out_fig = Output()
out = Output()

# Button
button = Button(description="Draw Sample", button_style='success')

def update_plots():
  with out_fig:
    out_fig.clear_output(wait=True)
    fig, (ax_mean, ax_std, ax_norm) = plt.subplots(1, 3, figsize=(15, 5))
    plt.subplots_adjust(wspace=0.4)
    ax_mean.set_title('Distribution of Sample Means')
    ax_mean.set_xlabel("Sample mean")
    ax_mean.set_ylabel("Frequency")
    ax_std.set_title('Distribution of Sample Variances')
    ax_std.set_xlabel("Sample std")
    ax_std.set_ylabel("Frequency")
    ax_norm.set_title('Distribution of Normalized Sample Means')
    ax_norm.set_xlabel("Normalized sample mean")
    ax_norm.set_ylabel("Frequency")

    ax_mean.hist(means, bins=20, alpha=0.7)
    ax_std.hist(vars, bins=20, alpha=0.7)
    ax_norm.hist(norm_means, bins=20, alpha=0.7)

    # fig.canvas.draw_idle()
    plt.show()

@button.on_click
def on_click(b):
  global means, vars, norm_means

  # Draw new sample
  sample = np.random.choice(charges, size=sample_size, replace=True)

  # Compute statistics
  mean = sample.mean()
  median = np.median(sample)
  q1, q3 = np.percentile(sample, [25, 75])
  var = sample.var(ddof=1)

  # Store for histograms
  means.append(mean)
  vars.append(var)
  norm_means.append((mean - full_sample_mean) / (full_sample_std / np.sqrt(sample_size)))

  # Update table
  with out:
      out.clear_output()
      display(pd.DataFrame({
          "Statistic": ["Mean", "Median", "Q1", "Q3", "Std"],
          "Value": [mean, median, q1, q3, var]
      }))

  # Update histograms
  update_plots()

# --- Layout ---
ui = VBox([button, out])
display(HBox([ui, out_fig]))


Maintenant que vous avez compris que chaque échantillon donne des valeurs de statistiques différentes, et que l'on peut donc étudier la distribution de ces statistiques comme on étudierait la distribution d'une variable aléatoire quelconque, nous allons pouvoir clôturer ce premier notebook en étudiant l'impact de la taille d'échantillon considérée les propriétés des statistiques.

Mais avant cela, posons-nous quelques questions:

❓Que représente chacune des statistiques étudiées ? Comment la moyenne d'échantillon normalisée se compare-t-elle à la moyenne d'échantillon ? Que signifie "normalisée" ? Est-ce que cette normalisation est possible en réalité ?

❓La façon dont ces statistiques se distribuent vous fait-elle penser à une/des
distributions connues ? Si oui, laquelle/lesquelles ?

❓L'écart-type d'échantillon qui est calculé est-il celui défini comme non-biaisé ou pas ? Que cela signifie-t-il ? Si l'on traçait la moyenne de ces écarts-types par dessus l'histogramme, vers quelle valeur convergerait-on ?

> 🧠 Écrivez ici ce que vous en pensez !


## Comportements asymptotiques 📈

Terminons en étudiant les propriétés asymptotiques des statistiques étudiées, c'est-à-dire les propriétés qu'ont ces statistiques lorsque la taille d'échantillon tend vers l'infini. Pour cela, considérons une autre variable de notre échantillon de départ : l'indice de masse corporelle (_BMI_).

Observons comment se distribue cette variable dans notre échantillon total, en comparaison avec les _charges_. Observons également comment les mêmes statistiques de sous-échantillon, pour ces deux variables différentes, se distribuent en fonction de la taille des sous-échantillons. 👀

Pour cela, vous pouvez sélectionner une taille de sous-échantillon et générer un grand nombre de ceux-ci à la fois (nombre que vous pouvez aussi sélectionner) afin d'en extraire les distributions de statistiques.

In [ ]:
charges = df['charges'].values
bmis = df['bmi'].values

# Parameters
true_mean_charges = charges.mean()
true_std_charges = charges.std(ddof=1)

true_mean_bmis = bmis.mean()
true_std_bmis = bmis.std(ddof=1)

# Widgets
sample_size = widgets.IntSlider(description='Sample size', min=5, max=500, step=5, value=50)
n_samples = widgets.IntText(description='N samples', value=200)
sample_btn = widgets.Button(description='Sample', button_style='success')

# Output
out_fig = widgets.Output()

def update_plots_(samples, sample_means, sample_vars, sample_norm_means, n_bins):
  with out_fig:
    out_fig.clear_output(wait=True)
    fig, ((ax_hist, ax_mean, ax_std, ax_norm), (ax2_hist, ax2_mean, ax2_std, ax2_norm)) = plt.subplots(2, 4, figsize=(30, 10))
    plt.subplots_adjust(wspace=0.4)

    ax_hist.set_title('Histogram')
    ax_hist.set_xlabel("Charges")
    ax_hist.set_ylabel("Density")
    ax_mean.set_title('Distribution of Sample Means')
    ax_mean.set_xlabel("Sample mean")
    ax_mean.set_ylabel("Frequency")
    ax_std.set_title('Distribution of Sample Variances')
    ax_std.set_xlabel("Sample variance")
    ax_std.set_ylabel("Frequency")
    ax_norm.set_title('Distribution of Normalized Means')
    ax_norm.set_xlabel("Normalized mean")
    ax_norm.set_ylabel("Frequency")

    ax2_hist.set_title('Histogram')
    ax2_hist.set_xlabel("BMI")
    ax2_hist.set_ylabel("Density")
    ax2_mean.set_title('Distribution of Sample Means')
    ax2_mean.set_xlabel("Sample mean")
    ax2_mean.set_ylabel("Frequency")
    ax2_std.set_title('Distribution of Sample Variances')
    ax2_std.set_xlabel("Sample variance")
    ax2_std.set_ylabel("Frequency")
    ax2_norm.set_title('Distribution of Normalized Means')
    ax2_norm.set_xlabel("Normalized mean")
    ax2_norm.set_ylabel("Frequency")

    ax_hist.hist(samples[0], bins=n_bins, alpha=0.7)
    ax_hist.axvline(x=true_mean_charges, color='r', linestyle='--')
    ax_mean.hist(sample_means[0], bins=n_bins, alpha=0.7)
    ax_mean.axvline(x=true_mean_charges, color='r', linestyle='--')
    ax_mean.axvline(x=sample_means[0].mean(), color='g', linestyle='--')
    ax_std.hist(sample_vars[0], bins=n_bins, alpha=0.7)
    ax_norm.hist(sample_norm_means[0], bins=n_bins, alpha=0.7)

    ax2_hist.hist(samples[-1], bins=n_bins, alpha=0.7)
    ax2_hist.axvline(x=true_mean_bmis, color='r', linestyle='--')
    ax2_mean.hist(sample_means[-1], bins=n_bins, alpha=0.7)
    ax2_mean.axvline(x=true_mean_bmis, color='r', linestyle='--')
    ax2_mean.axvline(x=sample_means[-1].mean(), color='g', linestyle='--')
    ax2_std.hist(sample_vars[-1], bins=n_bins, alpha=0.7)
    ax2_norm.hist(sample_norm_means[-1], bins=n_bins, alpha=0.7)

    plt.show()

@sample_btn.on_click
def on_sample_clicked(b):
    s = int(sample_size.value)
    n = int(n_samples.value)
    nbins = 25

    charges_samples = np.random.choice(charges, size=(n, s), replace=True)
    bmis_samples = np.random.choice(bmis, size=(n, s), replace=True)

    samples = charges_samples, bmis_samples

    sample_means = charges_samples.mean(axis=1), bmis_samples.mean(axis=1)
    sample_vars = charges_samples.var(axis=1, ddof=1), bmis_samples.var(axis=1, ddof=1)
    sample_norm_means = (sample_means[0] - true_mean_charges) / (true_std_charges / np.sqrt(s)), (sample_means[-1] - true_mean_bmis) / (true_std_bmis / np.sqrt(s))

    update_plots_((charges, bmis), sample_means, sample_vars, sample_norm_means, nbins)

controls = widgets.VBox([sample_size, n_samples, sample_btn])
ui = widgets.VBox([controls, out_fig], layout=widgets.Layout(align_items='flex-start'))
display(ui)

Bien-sûr, les deux variables ne sont pas distribuées de la même manière dans notre échantillon total, comme en attestent les histogrammes pour les deux variables.

❓En particulier, à quelles distributions les histogrammes générés vous font-ils penser ?

Au delà de ces différences, plusieurs comportements asymptotiques peuvent être observés. 🔍

❓Pour une taille d'échantillon faible (par exemple 5) et un nombre élevé d'échantillons (mettons 5000), comment les statistiques se distribuent-elles ? En particulier, la moyenne d'échantillon se distribue-t-elle de la même manière pour les deux variables considérées ? Quelle propriété est ici mise en évidence ?

❓Si vous augmentez la taille d'échantillon (par exemple 50), observez-vous toujours cette différence de distribution ? Pourquoi ? Quel comportement asymptotique cela montre-t-il ? Quel théorème est ici illustré ?

❓Vous pouvez observer deux lignes verticales (une rouge et une verte). Que représentent-elles ? Pour un faible nombre d'échantillons, ces deux lignes sont-elles proches ?  Et pour un grand nombre d'échantillon ? Quelle propriété est ici illustrée ? La taille des échantillons a-t-elle une influence sur cette propriété ?

❓Comment l'écart-type d'échantillon se distribue-t-il (n'hésitez pas à regarder comment la distribution évolue en partant d'une taille d'échantillons très faible pour progressivement augmenter) ? Est-ce en accord avec la théorie ?

❓Comment la moyenne normalisée se distribue-t-elle, en fonction de la taille d'échantillon et du nombre d'échantillons ? Si la  taille d'échantillon est faible, certains disent que cette variable peut être modélisée par une loi de Student-t. Est-ce correct d'affirmer cela ? Comment cette loi est-elle définie ? Y a-t-il une condition supplémentaire qui doit être vérifiée ? Qu'en est-il pour les deux variables considérées ici ?

> 🧠 Écrivez ici ce que vous en pensez !

## Conclusion et pistes de réflexion 💭

Vous voilà arrivés à la fin de ce premier notebook, bravo ! 🎉🥳

Nous y avons exploré différents concepts de statistiques descriptives, en résumant et visualisant des données afin de les décrire quantitativement et d'en extraire des tendances ("à quelle distribution cet histogramme me fait-il penser ?"). Bien que ce travail nous apporte déjà des informations précieuses, il ne nous permet pas de formellement quantifier l'incertitude ou de faire des prédictions à propos de données non rencontrées. 🤔

Le prochain notebook va donc se baser sur ces fondations en introduisant la notion de modèle statistique et de vraisemblance, qui fournissent un cadre théorique pour représenter les processus générateurs de données et estimer les paramètres qui expliquent le mieux nos observations. En substance, nous passons de la description de l'aspect des données à la modélisation de la manière dont elles auraient pu être générées ! 🚀